## MVTec LOCO AD

In [ ]:
from pathlib import Path

import numpy as np
from IPython.core.interactiveshell import InteractiveShell
from PIL import Image
from torchvision.transforms import ToPILImage

from anomalib.data.mvtec_loco import (
    MVTecLOCO,
    MVTecLOCODataset,
    download_and_extract_mvtec_loco,
)
from anomalib.pre_processing import PreProcessor
from anomalib.pre_processing.transforms import Denormalize

# make a cell print all the outputs instead of just the last one
InteractiveShell.ast_node_interactivity = "all"

# pylint: disable=locally-disabled, pointless-statement
# the ``pointless-statement`` warning is disabled because we use them to print stuff

# Download and extract the dataset

In [ ]:
root = Path("../../datasets/MVTecLOCO")
if not root.exists():
    download_and_extract_mvtec_loco(root)

### Torch Dataset

In [ ]:
MVTecLOCODataset??

To create `MVTecDataset` we need to import `pre_process` that applies transforms to the input image.

In [ ]:
PreProcessor??

In [ ]:
pre_process = PreProcessor(image_size=(100, 170), to_tensor=True)

#### Classification Task

In [ ]:
# MVTec LOCO Classification Train Set
mvtec_loco_dataset_classification_train = MVTecLOCODataset(
    root="../../datasets/MVTecLOCO",
    category="pushpins",
    split="train",
    pre_process=pre_process,
    task="classification",
)
mvtec_loco_dataset_classification_train.samples.head()

In [ ]:
sample = mvtec_loco_dataset_classification_train[0]
sample.keys()
sample["image"].shape

As can be seen above, when we choose `classification` task and `train` split, the dataset only returns `image`. This is mainly because training only requires normal images and no labels. Now let's try `test` split for the `classification` task

In [ ]:
# MVTec Classification Test Set
mvtec_loco_dataset_classification_test = MVTecLOCODataset(
    root="../../datasets/MVTecLOCO",
    category="pushpins",
    split="test",
    pre_process=pre_process,
    task="classification",
)

In [ ]:
sample = mvtec_loco_dataset_classification_test[0]
sample.keys()
sample["image"].shape
sample["image_path"]
sample["label"]
sample["super_anotype"], sample["anotype"]

Negative indices are also enabled.

In [ ]:
sample = mvtec_loco_dataset_classification_test[-1]
sample.keys()
sample["image"].shape
sample["image_path"]
sample["label"]
sample["super_anotype"], sample["anotype"]

#### Segmentation Task

It is also possible to configure the MVTec LOCO dataset for the segmentation task, where the dataset object returns image and ground-truth mask.

In [ ]:
# MVTec LOCO Segmentation Train Set
mvtec_loco_dataset_segmentation_train = MVTecLOCODataset(
    root="../../datasets/MVTecLOCO",
    category="pushpins",
    pre_process=pre_process,
    split="train",
    task="segmentation",
)
mvtec_loco_dataset_segmentation_train.samples.head()

In [ ]:
# MVTec LOCO Segmentation Test Set
mvtec_loco_dataset_segmentation_test = MVTecLOCODataset(
    root="../../datasets/MVTecLOCO",
    category="pushpins",
    pre_process=pre_process,
    split="test",
    task="segmentation",
)
sample = mvtec_loco_dataset_segmentation_test[20]
sample.keys()
sample["image"].shape
sample["mask"].shape

Let's visualize the image and the mask...

In [ ]:
img = ToPILImage()(Denormalize()(sample["image"].clone()))
msk = ToPILImage()(sample["mask"]).convert("RGB")

Image.fromarray(np.vstack((np.array(img), np.array(msk))))

### DataModule

So far, we have shown the Torch Dateset implementation of MVTec LOCO AD dataset. This is quite useful to get a sample, but we do need more than this when we train models in an end-to-end fashion.
 
The [PyTorch Lightning DataModule](https://pytorch-lightning.readthedocs.io/en/latest/data/datamodule.html) for MVTec LOCO AD (shown below) is handles the the dataset download, and train/val/test/inference dataloaders instantiation.

In [ ]:
MVTecLOCO??

In [ ]:
mvtec_datamodule = MVTecLOCO(
    root="../../datasets/MVTecLOCO",
    category="pushpins",
    image_size=(200, 340),  # (height, width) 5x smaller than original
    train_batch_size=32,
    test_batch_size=32,
    num_workers=8,
    task="segmentation",
)

# verify if the dataset is available and download it if not
mvtec_datamodule.prepare_data()

In [ ]:
# Train images

# instantiate the Torch Dataset(s), loading the (meta-)data into memory
mvtec_datamodule.setup("fit")

i, data = next(enumerate(mvtec_datamodule.train_dataloader()))
data.keys()
data["image"].shape

In [ ]:
# Validation images
mvtec_datamodule.setup("validate")
i, data = next(enumerate(mvtec_datamodule.val_dataloader()))
data.keys()
data["image"].shape
data["mask"].shape
data["super_anotype"][0], data["anotype"][0]

In [ ]:
# Test images
mvtec_datamodule.setup("test")
# iterate a few times so we can find a sample with an anomaly
for i, data in enumerate(mvtec_datamodule.test_dataloader()):
    if i == 5:
        break
data.keys()
data["image"].shape
data["mask"].shape
data["super_anotype"][0], data["anotype"][0]

img = ToPILImage()(Denormalize()(data["image"][0].clone()))
msk = ToPILImage()(data["mask"][0]).convert("RGB")

Image.fromarray(np.vstack((np.array(img), np.array(msk))))

TODO: show that the ground truth is divided in multiple images

TODO: create issue to correct docs in mvtec, e.g. it should not give example in the docstring but send the user
      to the notebooks (more maintainable)

As can be seen above, creating the dataloaders are pretty straghtforward, which could be directly used for training/testing/inference.